# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

#### Run shared detector on bash
```bash
python3 /home/kiro-ros/Projects/rnb-planning/src/scripts/demo_202107/demo_utils/shared_detector.py
```

#### Check and request ip setting from mobile udp client (robot-side)

#### 0.1.1 Set parameters and create planning scene

In [1]:
import numpy as np

CONNECT_CAM = False
CONNECT_INDY = False
CONNECT_MOBILE = False

CONNECT_TASK_PLANNER = False
VISUALIZE = False
VERBOSE = False
PLANNING_MULTIPROC = True
N_AGENTS = 10
NODE_TRIAL_MAX = N_AGENTS * 2
MAX_SOL_NUM = 5
BASE_COST_CUT = 110

TIMEOUT_MOTION = 0.5
TIMEOUT_FULL = 5

ROS_MASTER_ON_MOBILE = False
# Tool dimensions
TOOL_DIM = [0.15, 0.32]
TOOL_THICKNESS = 0.05
MARGIN = 0
TRACK_THICKNESS = 0.001

INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

COL_COLOR = (1,1,1,0.2)

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
if CONNECT_TASK_PLANNER:
    from demo_proto.DisinfectionOperationServicer import serve_on_thread
    servicer = serve_on_thread()

In [3]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()

from pkg.detector.multiICP.multiICP import *
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *
from demo_utils.area_select import DATASET_DIR, SweepDirections
from demo_utils.demo_config import *
from demo_utils.detection_util import *

from pkg.utils.shared_function import *
clear_channels_on("kiromobilemap")

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.KIRO_TOOL_PORT = '/dev/ttyS10'
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7kiro, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform")
ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

s_builder = SceneBuilder(None)
SceneBuilder.autostart_roscore = not ROS_MASTER_ON_MOBILE
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)
gscene.set_workspace_boundary(
    3, 9.5, -3.5, 0, -CLEARANCE, 3, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=CAM_LINK, center=(-0.0785, 0, 0.073))

add_brush(gscene, face_name=TOOL_NAME, tool_link=TIP_LINK,
          thickness=TOOL_THICKNESS, tool_dim=TOOL_DIM,
          col_color=COL_COLOR)

HOME_POSE = crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, 
                      incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

mplan.reset_PRQdict(enable_PRQ=0.5, radii=5e-2)
for tip_dir, SWEEP_AXIS in [
    (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)+"-PRQ.pkl"
    PRQ_PATH = os.path.join(SWEEP_DATA_DIR, filename)
    try:
        Pos_Rotvec_Qlist_dict = load_pickle(PRQ_PATH)
        mplan.register_PRQ(ROBOT_NAME, Pos_Rotvec_Qlist_dict, decimal=2)
        print("Loaded: {}".format(filename))
    except:
        print("File not exist: {}".format(filename))
        continue

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=NODE_TRIAL_MAX)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, \
    Gripper2Tool, PlacePlane, SweepFramer

brush_face = pscene.create_binder(
    bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
    point=(0,0, -gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE), 
    rpy=(0,0,0))

gscene.create_safe(
    gtype=GEOTYPE.BOX, name="floor_box", link_name="base_link",
    dims=(15,15,0.4), center=(0,0,0), rpy=(0,0,0), 
    color=(1, 1, 1, 0.1), display=True, collision=False, fixed=True)

gscene.add_highlight_axis("hl", "base_coord", T=np.identity(4), dims=(0.5,0.1,0.1))

kmb = crob.robot_dict["kmb0"]
indy = crob.robot_dict["indy1"]
mobile_box = gscene.NAME_DICT['kmb0_platform_Box_2']
crob.simulator.set_gscene(gscene)

if CONNECT_MOBILE:
    assert np.sum(np.abs(get_xyzw_cur()))>1e-4, "UDP Server not connected"
    
if CONNECT_CAM:
    realsense = RealSense()

from demo_utils.data_reconstructed_camera import DataRecontructedCamera
dcam = DataRecontructedCamera(crob, viewpoint)

connection command:
kmb0: True
indy1: False
==== Kiro Tool connected to /dev/ttyS10 (115200) ====
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Loaded: indy7kiro-frontX-PRQ.pkl
Loaded: indy7kiro-upZ-PRQ.pkl
Loaded: indy7kiro-downZ-PRQ.pkl


### Set UI

In [4]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


#### [DataRecon]

In [5]:
from demo_utils.data_reconstructed_camera import DataRecontructedCamera
dcam = DataRecontructedCamera(crob, viewpoint)

if not CONNECT_CAM:
    dcam.initialize()

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Exception in thread Thread-17:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/rnb/.local/lib/python2.7/site-packages/dash/dash.py", line 1716, in run_server
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/home/rnb/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1052, in run_simple
    inner()
  File "/home/rnb/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1005, in inner
    fd=fd,
  File "/home/rnb/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 848, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/home/rnb/.local/lib/python2.7/site-p

#### 0.1.2 Load environment map

In [6]:
from demo_utils.ros_map_utils import KiroMobileMap
kmm = KiroMobileMap(MOBILE_IP, IP_CUR, CONNECT_MOBILE)
            
VALID_BOX_SCALE = 0.8
VALID_SCORE_CUT = 50
kmb.coster = (lambda Q: 
                  np.max(
                      kmm.get_box_costs(mobile_box, Q, kmm.T_bi, kmm.cost_im, kmm.resolution, 
                                        scale=VALID_BOX_SCALE)))
kmb.cost_cut = VALID_SCORE_CUT
kmb.gscene = gscene

kmm.init_node(timeout=10)

In [7]:
pole_pt_list, pole_res = kmm.update_map(gscene, crob, MOBILE_BASE, timeout=100)
if not CONNECT_MOBILE:
    crob.joint_move_make_sure(kmm.Q_map)

[WARN] Vertices for mesh should be have center point (0,0,0). Auto adjusting.
End up at=[ 6.826 -1.716 -2.299] (0.0 / 0.0)


## 1. Prepare scene

In [8]:
TABLE_DIMS = (0.76,1.2, 0.5)
TABLE_LOC = (4.9,-3.0,TABLE_DIMS[2]/2)
TABLE_ANG = np.pi/2
TABLE_MARGIN_I = 0.03
TABLE_MARGIN_O = 0.03
CLEARANCE = 0.002
TABLE_COLOR = (0.8,0.8,0.8,1)
SURF_COLOR = (0,1,0,0.4)
COL_COLOR = (1,1,1,0.2)

gscene.create_safe(GEOTYPE.BOX, "table", link_name="base_link",
                   dims=TABLE_DIMS, center=TABLE_LOC, rpy=Rot2rpy(Rot_axis(3, TABLE_ANG)),
                   color=TABLE_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "table_col", link_name="base_link",
                   dims=np.add(TABLE_DIMS, 
                               (TABLE_MARGIN_O*2, TABLE_MARGIN_O*2, 0)), 
                   center=(0,0,0), rpy=(0,0,0), color=COL_COLOR, 
                   collision=True, parent="table"
                  )
table_surf = gscene.create_safe(GEOTYPE.BOX, "table_surf", link_name="base_link",
                   dims=np.subtract(TABLE_DIMS[:2]+(CLEARANCE,), 
                                    (TABLE_MARGIN_I*2, TABLE_MARGIN_I*2, 0)), 
                   center=(0,0,(TABLE_DIMS[2]+CLEARANCE)/2), 
                   rpy=(0,0,0), color=SURF_COLOR, parent="table",
                   collision=False
                  )

In [9]:
SOFA_DIMS = (0.56,1.2, 0.3)
SOFA_LOC = (7.6,-2.8,SOFA_DIMS[2]/2)
SOFA_SIDE_THIC = 0.2
SOFA_SIDE_HEIT = 0.2
SOFA_ANG = np.pi/2

SOFA_MARGIN_O = 0.03
SOFA_MARGIN_I = 0.05
SOFA_UV_DIST = 0.15

SOFA_COLOR = (0.4,0.4,0.4,1)

sofa = gscene.create_safe(GEOTYPE.BOX, "sofa", link_name="base_link",
                   dims=SOFA_DIMS, center=SOFA_LOC, 
                   rpy=Rot2rpy(Rot_axis(3, SOFA_ANG)),
                   color=SOFA_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_col", link_name="base_link",
                   dims=np.add(SOFA_DIMS, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa"
                  )
sofa_left = gscene.create_safe(GEOTYPE.BOX, "sofa_left", link_name="base_link",
                   dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(0,(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_left_col", link_name="base_link",
                   dims=np.add(sofa_left.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_left"
                  )
sofa_right = gscene.create_safe(GEOTYPE.BOX, "sofa_right", link_name="base_link",
                   dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(0,-(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_right_col", link_name="base_link",
                   dims=np.add(sofa_right.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_right"
                  )
sofa_back = gscene.create_safe(GEOTYPE.BOX, "sofa_back", link_name="base_link",
                   dims=(SOFA_SIDE_THIC, SOFA_DIMS[1]+SOFA_SIDE_THIC*2, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(-(SOFA_DIMS[0]+SOFA_SIDE_THIC)/2, 0, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_back_col", link_name="base_link",
                   dims=np.add(sofa_back.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_back"
                  )

sofa_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_surf", link_name="base_link",
                               dims=tuple(np.subtract(SOFA_DIMS[:2], SOFA_MARGIN_I*2))+(CLEARANCE,), 
                               center=(0,0,SOFA_DIMS[2]/2+SOFA_UV_DIST), collision=False, 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa"
                              )

sofa_back_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_back_surf", link_name="base_link",
                               dims=sofa_back.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(SOFA_MARGIN_I*2.5,0,sofa_back.dims[2]/2+SOFA_MARGIN_O+CLEARANCE), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_back"
                              )

sofa_left_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_left_surf", link_name="base_link",
                               dims=sofa_left.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(0,0,sofa_left.dims[2]/2+SOFA_UV_DIST), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_left"
                              )

sofa_right_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_right_surf", link_name="base_link",
                               dims=sofa_right.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(0,0,sofa_right.dims[2]/2+SOFA_UV_DIST), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_right"
                              )

sofa_front = gscene.create_safe(GEOTYPE.BOX, "sofa_front", link_name="base_link",
                               dims=(sofa.dims[2], sofa.dims[1])+(CLEARANCE,), 
                               center=(sofa.dims[0]/2+SOFA_UV_DIST,0,SOFA_MARGIN_I), collision=False, 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa"
                              )

sofa_left_front = gscene.create_safe(GEOTYPE.BOX, "sofa_left_front", link_name="base_link",
                               dims=(sofa_left.dims[2], sofa_left.dims[1])+(CLEARANCE,), 
                               center=(sofa_left.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_left"
                              )

sofa_right_front = gscene.create_safe(GEOTYPE.BOX, "sofa_right_front", link_name="base_link",
                               dims=(sofa_right.dims[2], sofa_right.dims[1])+(CLEARANCE,), 
                               center=(sofa_right.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_right"
                              )

sofa_back_front = gscene.create_safe(GEOTYPE.BOX, "sofa_back_front", link_name="base_link",
                               dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_SIDE_THIC*2-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               center=(sofa_back.dims[0]/2+SOFA_UV_DIST,0,SOFA_DIMS[2]/2+SOFA_MARGIN_I*1.5), 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, collision=False, parent="sofa_back"
                              )

### 1.0 Wait task start queue

In [10]:
if CONNECT_TASK_PLANNER:
    while servicer.object_info_running.object_id < 0:
        time.sleep(1)


## 2. Prepare cleaning

In [11]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *
from pkg.utils.code_scraps import get_look_motion

In [12]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
HOME_POSE_SWEEP = np.copy(Qcur)
# HOME_POSE_SWEEP[6:] = 0
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, 
                           gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    

VEL_LIMS = 0.2
ACC_LIMS = 0.2
RADI_DEG = 1

if CONNECT_INDY:
    indy.QVEL_LEVEL = 3
    indy.collision_policy = POLICY_NO_COLLISION_DETECTION
    indy.reset()

gtimer = GlobalTimer()
    
mode_switcher=ModeSwitcherLED(pscene, robot_name=ROBOT_NAME, brush_face=brush_face.geometry)

def no_offset(gxter, crob, mplan, robot_name, Qref):
    return Qref, Qref

## 3. Table cleaning

In [13]:
T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_HEIGHT = round(table_surf.get_tf(HOME_DICT)[2,3] + table_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]

In [14]:
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(table_surf, "front", "X", EE_HEIGHT, div_num=(4,4))
gxter.init_base_divs(Qcur)
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
gscene.clear_virtuals()

('Height Reference: ', 0.11400000277161598)
[INFO] Approach through: [ 4.97 -1.87 -1.57] (41.0) -> [ 5.   -2.   -1.57] (102.0)
End up at=[ 4.975 -1.875 -1.571] (2.22e-16 / 2.22e-16)
End up at=[ 4.997 -2.005 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((0.995, -0.097, -0.501), (0.0, 0.0, 1.0, 0.0)) / ((0.99, -0.1, -0.5), (1.0, 0.0))
[PLAN] Qcur: [ 4.997 -2.005 -1.571]
[PLAN] Qref: [ 4.997 -2.005 -1.571]
[PLAN] tar: [ 4.997 -2.005 -1.571]
End up at=[ 4.997 -2.005 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [0, 4, 8, 12]
[PLAN] Try idc (0, 4, 8, 12)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 0: required answers acquired  (0.8/5.0) ===============================
======================= terminated 3: Stop called from other agent  (0.8/5.0) ===============================
======================= terminated 6: Stop called from other agent  (0.8/5.0) ===============================
Goal reached

======================= terminated 1: required answers acquired  (0.9/5.0) ===============================
Goal reached
======================= terminated 8: required answers acquired  (1.0/5.0) ===============================
======================= terminated 2: Stop called from other agent  (1.0/5.0) ===============================
======================= terminated 3: Stop called from other agent  (1.0/5.0) ===============================
======================= terminated 7: Stop called from other agent  (1.0/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (1.0/5.0) ===============================
Goal reached
======================= terminated 9: required answers acquired  (1.1/5.0) ===============================
Goal reached
======================= terminated 6: required answers acquired  (1.1/5.0) ===============================
Goal reached
======================= terminated 0: required answers acquired  (1.2/

## 4. Sofa cleaning

### 4.1 upside

In [15]:
T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)

In [16]:
EE_HEIGHT = round(sofa_surf.get_tf(HOME_DICT)[2,3] + sofa_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_surf, "front", "X", EE_HEIGHT)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

('Height Reference: ', 0.06300000277161605)
[INFO] Depart via: [ 4.74 -1.85 -1.57] (41.0) <- [ 5.   -2.   -1.57] (102.0)
End up at=[ 4.738 -1.846 -1.571] (0.0 / 0.0)
[INFO] Approach through: [ 7.22 -1.88 -1.57] (41.0) -> [ 7.34 -1.93 -1.57] (66.0)
End up at=[ 7.22  -1.885 -1.571] (2.22e-16 / 2.22e-16)
End up at=[ 7.34  -1.928 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((0.872, 0.26, -0.45), (0.0, 0.0, 1.0, 0.0)) / ((0.87, 0.26, -0.45), (1.0, 0.0))
[PLAN] Qcur: [ 7.34  -1.928 -1.571]
[PLAN] Qref: [ 7.34  -1.928 -1.571]
[PLAN] tar: [ 7.34  -1.928 -1.571]
End up at=[ 7.34  -1.928 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [0, 4]
[PLAN] Try idc (0, 4)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 5: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 2: required answers acquired  (0.8/5.0) ===============================
Goal re

======================= terminated 1: required answers acquired  (0.7/5.0) ===============================
======================= terminated 6: Stop called from other agent  (0.8/5.0) ===============================
Goal reached
Goal reached
======================= terminated 3: required answers acquired  (0.8/5.0) ===============================
======================= terminated 9: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 0: Stop called from other agent  (0.8/5.0) ===============================
======================= terminated 2: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 8: required answers acquired  (0.8/5.0) ===============================
======================= terminated 5: Stop called from other agent  (0.9/5.0) ===============================
======================= terminated 4: Stop called from other agent  (0.9/5.0) =====

In [17]:
EE_HEIGHT = round(sofa_back_surf.get_tf(HOME_DICT)[2,3] + sofa_back_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_back_surf, "front", "X", EE_HEIGHT)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

('Height Reference: ', 0.145000002771616)
End up at=[ 7.44  -1.923 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((1.132, 0.16, -0.532), (0.0, 0.0, 1.0, 0.0)) / ((1.13, 0.16, -0.53), (1.0, 0.0))
[PLAN] Qcur: [ 7.44  -1.923 -1.571]
[PLAN] Qref: [ 7.44  -1.923 -1.571]
[PLAN] tar: [ 7.44  -1.923 -1.571]
End up at=[ 7.44  -1.923 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [1]
[PLAN] Try idc (1,)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 2: required answers acquired  (0.3/5.0) ===============================
======================= terminated 5: Stop called from other agent  (0.4/5.0) ===============================
======================= terminated 6: Stop called from other agent  (0.4/5.0) ===============================
======================= terminated 8: Stop called from other agent  (0.4/5.0) ===============================
======================= terminated 7: Stop called from other a

======================= terminated 5: required answers acquired  (0.5/5.0) ===============================
======================= terminated 8: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 1: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 9: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 6: required answers acquired  (0.5/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (0.5/5.0) ===============================
Goal reached
Goal reached
======================= terminated 2: required answers acquired  (0.5/5.0) ===============================
Goal reached
======================= terminated 3: required answers acquired  (0.5/5.0) ===============================
======================= terminated 7: required answers acquired  (0.5/5.0) ========

======================= terminated 8: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 9: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 0: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 4: Stop called from other agent  (0.6/5.0) ===============================
======================= terminated 3: Stop called from other agent  (0.6/5.0) ===============================
======================= terminated 2: Stop called from other agent  (0.6/5.0) ===============================
========================== FINISHED (0.7 / 5.0 s) ==============================]
Goal reached
[SIMUL] indy1 start tracking
LED ON
[SIMUL] indy1 start tracking
LED OFF
[SIMUL] indy1 start tracking
[SIMUL] indy1 stop tracking



In [18]:
EE_HEIGHT = round(sofa_right_surf.get_tf(HOME_DICT)[2,3] + sofa_right_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_right_surf, "front", "X", EE_HEIGHT)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

('Height Reference: ', 0.263000002771616)
End up at=[ 7.66  -1.908 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((0.892, 0.64, -0.65), (0.0, 0.0, 1.0, 0.0)) / ((0.89, 0.64, -0.65), (1.0, 0.0))
[PLAN] Qcur: [ 7.66  -1.908 -1.571]
[PLAN] Qref: [ 7.66  -1.908 -1.571]
[PLAN] tar: [ 7.66  -1.908 -1.571]
End up at=[ 7.66  -1.908 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [0, 1]
[PLAN] Try idc (0, 1)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 1: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 2: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 6: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (0.9/5.0) ===============================
Goal reached
===============

In [19]:
EE_HEIGHT = round(sofa_left_surf.get_tf(HOME_DICT)[2,3] + sofa_left_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_left_surf, "front", "X", EE_HEIGHT)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

('Height Reference: ', 0.263000002771616)
End up at=[ 7.34  -1.788 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((1.012, -0.44, -0.65), (0.0, 0.0, 1.0, 0.0)) / ((1.01, -0.44, -0.65), (1.0, 0.0))
[PLAN] Qcur: [ 7.34  -1.788 -1.571]
[PLAN] Qref: [ 7.34  -1.788 -1.571]
[PLAN] tar: [ 7.34  -1.788 -1.571]
End up at=[ 7.34  -1.788 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [0, 1]
[PLAN] Try idc (0, 1)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 9: required answers acquired  (0.8/5.0) ===============================
======================= terminated 3: Stop called from other agent  (0.9/5.0) ===============================
======================= terminated 6: Stop called from other agent  (0.9/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (0.9/5.0) ===============================
======================= terminated 5: Stop cal

In [20]:
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_back_front, "down", "Z", None)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

End up at=[ 7.357 -1.733 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((0.475, 0.243, 1.197), (0.707, -0.0, -0.707, 0.0)) / ((0.47, 0.24, 1.2), (0.71, 0.0))
[PLAN] Qcur: [ 7.357 -1.733 -1.571]
[PLAN] Qref: [ 7.357 -1.733 -1.571]
[PLAN] tar: [ 7.357 -1.733 -1.571]
End up at=[ 7.357 -1.733 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [0]
[PLAN] Try idc (0,)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 0: required answers acquired  (0.5/5.0) ===============================
======================= terminated 3: required answers acquired  (0.5/5.0) ===============================
Goal reached
======================= terminated 2: required answers acquired  (0.5/5.0) ===============================
======================= terminated 5: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 7: Stop called from other agent  (0.5/5.0) =

Process Process-163:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()


Goal reached


Process Process-162:


Goal reached


...

    **kwargs)


  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 290, in test_connection
    self.mplan.plan_transition(from_state, to_state, verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/interface.py", line 153, in plan_transition
    verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/moveit/moveit_planner.py", line 385, in plan_algorithm


    from_state.Q[self.combined_robot.idx_dict[rname]]
TypeError: list indices must be integers, not tuple


======================= terminated 7: Stop called from other agent  (0.6/5.0) ===============================
======================= terminated 2: Stop called from other agent  (0.6/5.0) ===============================
======================= terminated 4: Stop called from other agent  (0.7/5.0) ===============================
======================= terminated 5: Stop called from other agent  (0.7/5.0) ===============================
======================= terminated 3: Stop called from other agent  (0.7/5.0) ===============================
========================== FINISHED (0.8 / 5.0 s) ==============================]
[PLAN] Line idc [2]
[PLAN] Try idc (2,)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 2: required answers acquired  (0.4/5.0) ===============================
======================= terminated 5: Stop called from other agent  (0.4/5.0) ===============================
================

========================== FINISHED (0.7 / 5.0 s) ==============================]
Goal reached
[SIMUL] indy1 start tracking
LED ON
[SIMUL] indy1 start tracking
LED OFF
[SIMUL] indy1 start tracking
LED ON
[SIMUL] indy1 start tracking
LED OFF
[SIMUL] indy1 start tracking
LED ON
[SIMUL] indy1 start tracking
LED OFF
[SIMUL] indy1 start tracking
LED ON
[SIMUL] indy1 start tracking
LED OFF
[SIMUL] indy1 start tracking
[SIMUL] indy1 stop tracking



In [21]:
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_right_front, "down", "Z", None)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

End up at=[ 7.51  -1.716 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((0.3, 0.79, 0.654), (0.707, -0.0, -0.707, 0.0)) / ((0.3, 0.79, 0.65), (0.71, 0.0))
[PLAN] Qcur: [ 7.51  -1.716 -1.571]
[PLAN] Qref: [ 7.51  -1.716 -1.571]
[PLAN] tar: [ 7.51  -1.716 -1.571]
End up at=[ 7.51  -1.716 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [0, 1]
[PLAN] Try idc (0, 1)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 0: required answers acquired  (0.7/5.0) ===============================
======================= terminated 4: Stop called from other agent  (0.7/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (0.7/5.0) ===============================
Goal reached
======================= terminated 7: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 8: required answers acquired 

In [ ]:
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_front, "down", "Z", None)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

End up at=[ 7.493 -1.453 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Adjust base once. ((0.2, 0.107, 0.917), (0.707, -0.0, -0.707, 0.0)) / ((0.2, 0.11, 0.92), (0.71, 0.0))
[PLAN] Qcur: [ 7.493 -1.453 -1.571]
[PLAN] Qref: [ 7.493 -1.453 -1.571]
[PLAN] tar: [ 7.493 -1.453 -1.571]
End up at=[ 7.493 -1.453 -1.571] (2.22e-16 / 2.22e-16)
[PLAN] Line idc [0]
[PLAN] Try idc (0,)
Use 10/36 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 3: required answers acquired  (0.8/5.0) ===============================
======================= terminated 1: required answers acquired  (0.8/5.0) ===============================
======================= terminated 7: Stop called from other agent  (0.8/5.0) ===============================
======================= terminated 9: Stop called from other agent  (0.8/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (0.8/5.0) ===

======================= terminated 5: node queue empty 4th  (2.1/5.0) ===============================
======================= terminated 0: node queue empty 5th  (2.1/5.0) ===============================
======================= terminated 4: node queue empty 6th  (2.3/5.0) ===============================
======================= terminated 6: node queue empty 7th  (2.3/5.0) ===============================
======================= terminated 7: node queue empty 8th  (2.4/5.0) ===============================
======================= terminated 9: node queue empty 9th  (2.5/5.0) ===============================
======================= terminated 8: node queue empty 10th  (2.6/5.0) ===============================
========================== FINISHED (2.6 / 5.0 s) ==============================]
[PLAN] Line idc [3]
[PLAN] Try idc (3,)
Use 10/36 agents
======================= terminated 2: node queue empty 1th  (2.0/5.0) ===============================
======================= terminated 3: node q

======================= terminated 9: required answers acquired  (0.7/5.0) ===============================
======================= terminated 2: required answers acquired  (0.7/5.0) ===============================
======================= terminated 3: Stop called from other agent  (0.7/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (0.8/5.0) ===============================
======================= terminated 1: Stop called from other agent  (0.9/5.0) ===============================
========================== FINISHED (0.9 / 5.0 s) ==============================]
[PLAN] Line idc [2]
[PLAN] Try idc (2,)
Use 10/36 agents
======================= terminated 4: node queue empty 1th  (1.9/5.0) ===============================
======================= terminated 0: node queue empty 2th  (2.0/5.0) ===============================
======================= terminated 1: node queue empty 3th  (2.1/5.0) ===============================
=

[SIMUL] indy1 start tracking


In [ ]:
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur)
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_left_front, "down", "Z", None)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

### Finish

In [ ]:
if CONNECT_TASK_PLANNER:
    servicer.mark_task_finished()

* MAX_SOL_NUM 적용 v
* 위 오른쪽 안닦이는 문제 파악 - guardrail 마진 - v
* 아래 닦을 때 위쪽 가운데 바운더리 추가 - v
* 침대 닦을 때 옷장 바운더리 마진 추가 - v
* 위치 보정, 드리프트 코드 검토 - ok - v
* 인디 툴 무게 v
* 툴 조인트 플래닝에 추가?

* Adjust에서 Qref와 Qcur 분리